# DATA ANALYSIS (RACERS)
In this notebook we are going to asses the quality of the racers dataset provided for the course project


In [5]:
import pandas as pd
from os import path
datasetname=path.join('dataset', 'cyclists.csv')
df=pd.read_csv(datasetname)

## WRONG DATA
To asses which are the wrong data in the dataset we have to check the data type in the dataset.

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6134 entries, 0 to 6133
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   _url         6134 non-null   object 
 1   name         6134 non-null   object 
 2   birth_year   6121 non-null   float64
 3   weight       3078 non-null   float64
 4   height       3143 non-null   float64
 5   nationality  6133 non-null   object 
dtypes: float64(3), object(3)
memory usage: 287.7+ KB


Just from the info command we can start to see the first problems, the url, name and nationality should be string types, the birth_year should be an Integer type, weight and height are ok to be float64.
Now we can show every element which don't respect the natural type to check what are the problems.

First we check if the object is an inference choice for string values or if there are some kind of null pointer or not accepted values

In [8]:
def wrong_type(valore, tipo_atteso):
    try:
        return not isinstance(valore, tipo_atteso)
    except:
        return True  

### _url

In [27]:
column = '_url'
wrong_elements = df[df[column].apply(wrong_type, tipo_atteso=str)]

print(wrong_elements[column])

Series([], Name: _url, dtype: object)


As we can see all the elements are str type. Since the url are in the form "name-surname" we can check if for all the rows this rule is respected.

In this case we can see that the main differences between the two column is the replacing of the double whitespace with a '-' (name-surname), the replace of the singular withespace with a '-' (first name-second name) and the standardization of the letters with accent in the base form (è in e) .

In [20]:
import unicodedata

def remove_accent(text):
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')
    return text

differences = df[df['name'].apply(remove_accent).str.lower().str.replace('  ', '-').str.replace(' ','-') != df['_url']]

print(differences[['name', '_url']])

                         name                            _url
19             Michał  Paluta                   michal-paluta
25        Graeme Allen  Brown                    graeme-brown
32           Piotr  Przydział                 piotr-przydzial
49           Jesús  Rodríguez       jesus-rodriguez-rodriguez
68              Ivan  Herrero           iban-herrero-atienzar
...                       ...                             ...
6093  Raúl Alexander  Montaña  raul-alexander-montana-herrera
6112     Juan Carlos  Jusdado      juan-carlos-jusdado-ibanez
6113           Joseba  Albizu            joseba-albizu-lizaso
6121         Jacob  Hindsgaul          jacob-hindsgaul-madsen
6129      Juan José  Martínez         juan-jose-martinez-diaz

[618 rows x 2 columns]


From this analysis we can see that there are 618 rows which don't respect the main conversion rules, some with missin elements (second name or surname) and some with wrong characters. In some cases in the '_url' there are more information in respect of the 'name' column, because of that it's better to split the column 'name' in 'name' and 'surname' ('secondname' could cause too many empty elements) and rebuild the full name of every rider thanks to the column with more information.

In [28]:
indici_nan = df.index[df[column].isna()].tolist()

print(f"Nan indexes '{column}': {indici_nan}")
print(f"Numbner of missing values '{column}': {len(indici_nan)}")

Nan indexes '_url': []


We have no missing values in this column

### name

In [30]:
column = 'name'
wrong_elements = df[df[column].apply(wrong_type, tipo_atteso=str)]

print(wrong_elements[column])

Series([], Name: name, dtype: object)


In [31]:
indici_nan = df.index[df[column].isna()].tolist()

print(f"Nan indexes '{column}': {indici_nan}")
print(f"Numbner of missing values '{column}': {len(indici_nan)}")

Nan indexes 'name': []


We have no missing values for this column

### birth_year

In [32]:
column = 'birth_year'
wrong_elements = df[df[column].apply(wrong_type, tipo_atteso=int)]

print(wrong_elements[column])

0       1964.0
1       1965.0
2       1996.0
3       1995.0
4       1997.0
         ...  
6129    1966.0
6130    1998.0
6131    1973.0
6132    1985.0
6133    1999.0
Name: birth_year, Length: 6134, dtype: float64


All the elements of birth year type are float, not int but for computation reason. Let's only check if there are elements with number after the last digit.


In [33]:
column = 'birth_year'
decilmal_elements = df[df[column] % 1 != 0]

print(decilmal_elements[column])


9      NaN
601    NaN
894    NaN
2408   NaN
2515   NaN
2536   NaN
3046   NaN
3551   NaN
4142   NaN
4384   NaN
4756   NaN
6072   NaN
6080   NaN
Name: birth_year, dtype: float64


We can see that all the birth year with a decimal part ar Nan so for this section of the analysis it's fine.

In [35]:
indici_nan = df.index[df[column].isna()].tolist()

print(f"Nan indexes '{column}': {indici_nan}")
print(f"Numbner of missing values '{column}': {len(indici_nan)}")

Nan indexes 'birth_year': [9, 601, 894, 2408, 2515, 2536, 3046, 3551, 4142, 4384, 4756, 6072, 6080]
Numbner of missing values 'birth_year': 13


### nationality

In [39]:
df['nationality'].head()

0          Italy
1         France
2    Netherlands
3        Belgium
4          Spain
Name: nationality, dtype: object

In [36]:
column = 'nationality'
wrong_elements = df[df[column].apply(wrong_type, tipo_atteso=str)]

print(wrong_elements[column])

9    NaN
Name: nationality, dtype: object


Before changing the type of the column we have to remove the Nan values but it's another study case so we leave in this state. Another verification we can do is if the nationality corresponds to a real state and there are no wrong entries, to do this we can use pycountry.

In [45]:
import pycountry
nazioni_valide = {country.name.lower() for country in pycountry.countries}

print("russian federation" in nazioni_valide) 

True


In [57]:
import pycountry

valid_nat = {country.name.lower() for country in pycountry.countries}

# Crea una lista per memorizzare le Not valid nationality
unvalid_nat = []

# Verifica le nazionalità nel DataFrame senza modificarlo
for nat in df['nationality']:
    if nat is not None:
        if str(nat).lower() not in valid_nat:
            unvalid_nat.append(nat)

# Conta le Not valid nationality
number_unvalid_nat = len(unvalid_nat)

# Visualizza i risultati
print(f"Number of not valid country: {number_unvalid_nat}")
if number_unvalid_nat > 0:
    print("Not valid nationality:")
    print(unvalid_nat)


Number of not valid country: 310
Not valid nationality:
[nan, 'Russia', 'Great Britain', 'Russia', 'Great Britain', 'Great Britain', 'Yugoslavia', 'Russia', 'Russia', 'Great Britain', 'Czech Republic', 'Czech Republic', 'Russia', 'Russia', 'Great Britain', 'Russia', 'Russia', 'Great Britain', 'Great Britain', 'Russia', 'Russia', 'Russia', 'Czech Republic', 'Russia', 'Great Britain', 'Great Britain', 'Russia', 'Great Britain', 'Great Britain', 'Venezuela', 'Russia', 'Russia', 'Russia', 'Great Britain', 'Russia', 'Russia', 'Great Britain', 'Russia', 'Russia', 'Russia', 'Russia', 'Great Britain', 'Czech Republic', 'Russia', 'Czech Republic', 'Great Britain', 'Russia', 'Russia', 'Russia', 'Great Britain', 'Great Britain', 'Taiwan', 'Czech Republic', 'Moldova', 'Russia', 'Russia', 'Russia', 'Great Britain', 'Russia', 'Czech Republic', 'Great Britain', 'Great Britain', 'Iran', 'Great Britain', 'Great Britain', 'Great Britain', 'Venezuela', 'Russia', 'Russia', 'Venezuela', 'Russia', 'Great Br

The not valid nation are in most cases real nation not considered by the library, we can add some exception to handle this case

In [58]:
# Dizionario di corrispondenza per nazionalità comuni non standard
corr_nat = {
    'russia': 'russian federation',
    'great britain': 'united kingdom',
    'yugoslavia': 'serbia',
    'czech republic': 'czechia',
    'taiwan': 'taiwan, province of china',
    'venezuela': 'venezuela, bolivarian republic of',
    'iran': 'iran, islamic republic of',
    'moldova': 'moldova, republic of',
}

# Crea un elenco per memorizzare le nazionalità non valide
unvalid_nat = []

for nat in df['nationality']:
    if nat is None:
        continue
    nat_lower = str(nat).lower()
    # Verifica se la nazionalità è nel dizionario di corrispondenza
    if nat_lower in corr_nat:
        nat_lower = corr_nat[nat_lower]
    
    if nat_lower not in valid_nat:
        unvalid_nat.append(nat)

# Conta le nazionalità non valide
number_unvalid_nat = len(unvalid_nat)

# Visualizza i risultati
print(f"Number of not valid nationality: {number_unvalid_nat}")
if number_unvalid_nat > 0:
    print("Not valid nationality:")
    print(unvalid_nat)

Number of not valid nationality: 2
Not valid nationality:
[nan, 'Hongkong']


The last nation which is still invalid is hong kong, in pycountry is 'hong kong' let's see if its the only one entry or if it's a typo

In [59]:
hong_kong = df[df['nationality'].str.lower() == 'hong kong']

print(hong_kong)

Empty DataFrame
Columns: [_url, name, birth_year, weight, height, nationality]
Index: []


It's not a typo so we can conclude that every country is valid.

In [60]:
indici_nan = df.index[df[column].isna()].tolist()

print(f"Nan indexes '{column}': {indici_nan}")
print(f"Numbner of missing values '{column}': {len(indici_nan)}")

Nan indexes 'nationality': [9]
Numbner of missing values 'nationality': 1


## DUPLICATE DATA
For this task we can check two things:

- Row which are the exact identical
- Row with the same _url (our identifier)

In [22]:
same_rows = df.duplicated(keep=False)  

url_duplicates = df.duplicated(subset=['_url'], keep=False) 

name_duplicates = df.duplicated(subset=['name'], keep=False) 


print(f"Number of identical row: {same_rows.sum()}")
print(f"Number of row with same _url: {url_duplicates.sum()}")
print(f"Number of row with same name: {name_duplicates.sum()}")

Number of identical row: 0
Number of row with same _url: 0
Number of row with same name: 14


We can see that the number of identical row is zero but we found 14 rows with same name, let's check them.

In [23]:
print(df[name_duplicates][['name', '_url']])

                    name                        _url
347        Andrea  Peron              andrea-peron-1
1745    Roman  Kreuziger          roman-kreuziger-sr
2235   Alessandro  Pozzi           alessandro-pozzi2
2601    Roman  Kreuziger             roman-kreuziger
2682       Andrea  Peron                andrea-peron
2862    Antonio  Cabello       antonio-cabello-baena
2939        Jesús  López               jesus-lopez23
2953  Alberto  Fernández     alberto-fernandez-sainz
3238    Antonio  Cabello             antonio-cabello
4917   Sergio  Domínguez  sergio-dominguez-rodriguez
4919   Sergio  Domínguez      sergio-dominguez-munoz
5040        Jesús  López          jesus-lopez-carril
5720  Alberto  Fernández    alberto-fernandez-blanco
5722   Alessandro  Pozzi            alessandro-pozzi


In this case the disambuigation is not so trivial, some times the duplicates are converted with a number after the classical url convertion, other ones it's converted with different information (second name). This makes the direct conversion harder but if we consider only the url as the identifier we have a well defined dataset in this case. Suggestion: redifinition of the url identifier in a clearly way :

Name Second Surname in name-second-surname(number if there are identical name)